In [1]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta, timezone
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

# ----------------------------------------------------------------------------------------
# Black-Scholes Option Pricing Function Parameters
#
# S     : float - Current price of the underlying stock (spot price)
#         Example: If AAPL is trading at $175, then S = 175
#
# K     : float - Strike price of the option contract
#         The fixed price at which the option holder can buy (call) or sell (put) the stock
#
# T     : float - Time to expiration in years
#         Example: If the option expires in 30 days, then T = 30 / 365 ≈ 0.0822
#
# r     : float - Risk-free interest rate (annual), expressed as a decimal
#         Example: If the rate is 5%, then r = 0.05
#
# sigma : float - Volatility of the underlying stock’s returns (annualized), expressed as a decimal
#         Usually implied volatility (IV). Example: 30% volatility → sigma = 0.30
#
# Returns:
#   Theoretical price of a European call or put option using the Black-Scholes model

def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def black_scholes_put_price(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(K - S, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def current_stock_price(ticker):
    url = f"https://eodhd.com/api/real-time/{ticker}.US?api_token={API_KEY}&fmt=json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return float(data.get("close", 0))
    except Exception as e:
        logging.warning(f"Failed to fetch current price for {ticker}: {e}")
        return 0

def fetch_filtered_options(symbol, exp_from, exp_to, strike_from, strike_to, option_type):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": symbol,
        "filter[type]": option_type,
        "filter[exp_date_from]": exp_from,
        "filter[exp_date_to]": exp_to,
        "filter[strike_from]": strike_from,
        "filter[strike_to]": strike_to,
        "sort": "exp_date",
        "page[limit]": 1000
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        return response.json().get("data", [])
    except Exception as e:
        logging.warning(f"Error fetching options contracts for {symbol}: {e}")
        return []


def analyze_options_unicorn(
    tickers,
    option_type="call",
    days_until_exp=90,
    strike_pct=0.2,
    days_to_gain=30,
    stock_gain_pct=0.1,
    risk_free_rate=0.05,
    allocation=None
):
    today = datetime.now(timezone.utc)
    eval_date = today + timedelta(days=days_to_gain)
    exp_from = (today + timedelta(days=days_until_exp - 30)).strftime("%Y-%m-%d")
    exp_to = (today + timedelta(days=days_until_exp + 30)).strftime("%Y-%m-%d")

    results = []

    for ticker in tickers:
        try:
            current_price = current_stock_price(ticker)
            if current_price <= 0:
                logging.info(f"Skipping {ticker}: invalid current price.")
                continue

            target_strike = current_price * (1 + strike_pct)
            lower = target_strike * 0.95
            upper = target_strike * 1.05

            options_data = fetch_filtered_options(ticker, exp_from, exp_to, lower, upper, option_type)
            if not options_data:
                logging.info(f"No options data returned for {ticker}")
                continue

            grouped = {}
            for opt in options_data:
                attr = opt.get("attributes", {})
                exp = attr.get("exp_date")
                if exp not in grouped:
                    grouped[exp] = []
                grouped[exp].append(opt)

            filtered_options = []
            for exp, group in grouped.items():
                closest = sorted(
                    group,
                    key=lambda x: abs(x.get("attributes", {}).get("strike", 0) - target_strike)
                )[:2]
                filtered_options.extend(closest)

            for opt in filtered_options:
                attr = opt.get("attributes", {})
                strike = attr.get("strike")
                exp_date_str = attr.get("exp_date")
                last_price = attr.get("last", 0)
                last_Premium = attr.get("last", 0)
                iv = attr.get("volatility", 0.3)

                delta = attr.get("delta")
                theta = attr.get("theta")
                gamma = attr.get("gamma")
                vega = attr.get("vega")
                rho = attr.get("rho")

                bid = attr.get("bid")
                bid_date = attr.get("bid_date")
                ask = attr.get("ask")
                ask_date = attr.get("ask_date")
                volume = attr.get("volume")

                if not strike or not exp_date_str:
                    continue

                if not last_price and bid is not None and ask is not None:
                    try:
                        last_price = (float(bid) + float(ask)) / 2
                    except:
                        last_price = 0

                exp_date = datetime.strptime(exp_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
                exp_days = (exp_date - today).days
                T_eval = max((exp_date - eval_date).days / 365, 0.0001)

                sim_stock = current_price * (1 + stock_gain_pct)

                if option_type == "call":
                    est_value = black_scholes_call_price(sim_stock, strike, T_eval, risk_free_rate, iv)
                    pct_otm_itm = (strike - current_price) / current_price * 100
                elif option_type == "put":
                    est_value = black_scholes_put_price(sim_stock, strike, T_eval, risk_free_rate, iv)
                    pct_otm_itm = (current_price - strike) / current_price * 100
                else:
                    est_value = None
                    pct_otm_itm = "NA"

                pct_gain = ((est_value - last_price) / last_price) * 100 if last_price and est_value else None
                alloc_value = allocation if allocation else None
                simulated_value = (alloc_value * (1 + pct_gain / 100)) if alloc_value and pct_gain is not None else None

                result = {
                    "Ticker": ticker,
                    "Option Type": option_type,
                    "Expiration": exp_date_str,
                    "Strike": strike,
                    "% OTM/ITM": round(pct_otm_itm, 2) if isinstance(pct_otm_itm, (int, float)) else pct_otm_itm,
                    "Underlying Price": round(current_price, 2),
                    "Simulated Underlying": round(sim_stock, 2),
                    "Current Premium": round(last_price, 2),
                    "Simulated Premium": round(est_value, 2) if est_value is not None else "NA",
                    "Days Until Expiration": exp_days,
                    "Days to Gain": days_to_gain,
                    "Underlying Gain %": round(stock_gain_pct * 100, 2),
                    "Premium % Gain": round(pct_gain, 2) if pct_gain is not None else "NA",
                    "Implied Volatility": round(iv * 100, 2),
                    "Delta": round(delta, 4) if delta is not None else "NA",
                    "Theta": round(theta, 4) if theta is not None else "NA",
                    "Gamma": round(gamma, 4) if gamma is not None else "NA",
                    "Vega": round(vega, 4) if vega is not None else "NA",
                    "Rho": round(rho, 4) if rho is not None else "NA",
                    "Bid": bid,
                    "Ask": ask,
                    "Last Premium": last_Premium,
                    "Bid Date": bid_date,
                    "Ask Date": ask_date,
                    "Volume": volume
                }

                if alloc_value is not None and simulated_value is not None:
                    result["Allocated Equity"] = round(alloc_value, 2)
                    result["Simulated Equity"] = round(simulated_value, 2)

                results.append(result)

        except Exception as e:
            logging.error(f"Error processing {ticker}: {e}")

    return pd.DataFrame(results)


def run_multiple_analyses(param_sets):
    combined_df = pd.DataFrame()
    for params in param_sets:
        label = params.pop("label", "")
        allocation = params.pop("allocation", None)
        df = analyze_options_unicorn(**params, allocation=allocation)
        df["Run Label"] = label
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df



def highlightsTable(df):
    cols = [
        "Ticker",
        "Option Type",
        "Expiration",
        "Strike",
        "Current Stock",
        "Current Premium",
        "Stock Gain %",
        "Premium % Gain",
        "Implied Volatility",
        "Bid Date",
        "Volume"
    ]
    return df[cols]

# # Example use
# param_sets = [
#     {
#         "tickers": ["AAPL"],
#         "option_type": "call",
#         "days_until_exp": 120,
#         "strike_pct": 0.2,
#         "days_to_gain": 30,
#         "stock_gain_pct": 0.1,
#         "allocation": 100,
#         "label": "Short-Term Conservative"
#     },
#     {
#         "tickers": ["AAPL"],
#         "option_type": "put",
#         "days_until_exp": 120,
#         "strike_pct": -0.2,
#         "days_to_gain": 30,
#         "stock_gain_pct": -0.1,
#         "allocation": 40,
#         "label": "Long-Term Aggressive (Put)"
#     }
# ]

# combined_results = run_multiple_analyses(param_sets)
# display(combined_results)
# # combined_results.to_csv("optionsTests.csv", index=False)


In [2]:
import requests
import json

# API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

def fetch_specific_option(symbol, strike, exp_date, option_type="call"):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": symbol,
        "filter[strike_eq]": strike,
        "filter[exp_date_eq]": exp_date,
        "filter[type]": option_type,
        "page[limit]": 1
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json().get("data", [])
        if not data:
            print(f"No option found for {symbol} {strike} {option_type.upper()} expiring {exp_date}")
            return

        contract = data[0].get("attributes", {})
        print(json.dumps(contract, indent=4))  # Pretty-print the full contract data

    except Exception as e:
        print(f"Error fetching option: {e}")

# Example usage
if __name__ == "__main__":
    fetch_specific_option(
        symbol="AAPL",
        strike=225,
        exp_date="2025-07-18",
        option_type="call"
    )


{
    "contract": "AAPL250718C00225000",
    "underlying_symbol": "AAPL",
    "exp_date": "2025-07-18",
    "expiration_type": "monthly",
    "type": "call",
    "strike": 225,
    "exchange": "NASDAQ",
    "currency": "USD",
    "open": 0.19,
    "high": 0.2,
    "low": 0.16,
    "last": 0.19,
    "last_size": 3,
    "change": -0.01,
    "pctchange": -5,
    "previous": 0.2,
    "previous_date": "2025-06-25",
    "bid": 0.19,
    "bid_date": "2025-06-26 19:59:59",
    "bid_size": 10,
    "ask": 0.2,
    "ask_date": "2025-06-26 19:59:59",
    "ask_size": 85,
    "moneyness": -0.12,
    "volume": 1026,
    "volume_change": -4249,
    "volume_pctchange": -80.55,
    "open_interest": 22213,
    "open_interest_change": -1015,
    "open_interest_pctchange": -4.37,
    "volatility": 0.251,
    "volatility_change": 0.0086,
    "volatility_pctchange": 3.53,
    "theoretical": 0.19,
    "delta": 0.038979,
    "gamma": 0.00681,
    "theta": -0.024548,
    "vega": 0.041642,
    "rho": 0.004608,
 

In [3]:
def simulate_option_watchlist_single(ticker, option_type, strike, expiration, days_to_gain,
                                     number_of_contracts, average_cost_per_contract, risk_free_rate=0.05):
    try:
        current_price = current_stock_price(ticker)
        if current_price <= 0:
            logging.warning("Invalid stock price for %s", ticker)
            return pd.DataFrame()

        # Fetch exact contract data from EODHD
        params = {
            "api_token": API_KEY,
            "filter[underlying_symbol]": ticker,
            "filter[type]": option_type,
            "filter[exp_date_from]": expiration,
            "filter[exp_date_to]": expiration,
            "filter[strike_from]": strike,
            "filter[strike_to]": strike,
            "page[limit]": 1
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json().get("data", [])

        if not data or "attributes" not in data[0]:
            logging.warning(f"No valid contract found for {ticker} {option_type} {strike} {expiration}")
            return pd.DataFrame()

        attr = data[0]["attributes"]
        last_price = float(attr.get("last", 0))
        iv = float(attr.get("volatility", 0.3))

        if last_price <= 0:
            logging.warning(f"Missing or invalid last premium for {ticker}")
            return pd.DataFrame()

        exp_date = datetime.strptime(expiration, "%Y-%m-%d").replace(tzinfo=timezone.utc)
        eval_date = datetime.now(timezone.utc) + timedelta(days=days_to_gain)
        T_eval = max((exp_date - eval_date).days / 365, 0.0001)

        total_cost = average_cost_per_contract * number_of_contracts * 100
        current_gain_pct = ((last_price - average_cost_per_contract) / average_cost_per_contract) * 100
        current_gain_dollar = (last_price - average_cost_per_contract) * number_of_contracts * 100

        # Define the % stock movement scenarios (added ±100% and ±200%)
        scenarios = [0.05, 0.10, 0.20, 0.50, 1.0, 2.0]
        rows = []

        for pct in scenarios:
            stock_up = current_price * (1 + pct)
            stock_down = current_price * (1 - pct)

            if option_type == "call":
                premium_up = black_scholes_call_price(stock_up, strike, T_eval, risk_free_rate, iv)
                premium_down = black_scholes_call_price(stock_down, strike, T_eval, risk_free_rate, iv)
            elif option_type == "put":
                premium_up = black_scholes_put_price(stock_up, strike, T_eval, risk_free_rate, iv)
                premium_down = black_scholes_put_price(stock_down, strike, T_eval, risk_free_rate, iv)
            else:
                continue

            pct_up_gain = ((premium_up - last_price) / last_price) * 100
            pct_down_loss = ((premium_down - last_price) / last_price) * 100

            rows.append({
                # Contract Info
                "Ticker": ticker,
                "Current Underlying": round(current_price, 2),
                "Option Type": option_type,
                "Strike": strike,
                "Expiration": expiration,
                "Implied Volatility": round(iv * 100, 2),

                # Position Details
                "Number of Contracts": number_of_contracts,
                "Average Cost per Contract": average_cost_per_contract,
                "Equity Invested": round(total_cost, 2),
                "Current Premium": round(last_price, 2),
                "Current Equity": round(last_price * number_of_contracts * 100, 2),
                "Current % Change": round(current_gain_pct, 2),

                # Scenario Setup
                "Days to Gain": days_to_gain,
                "Scenario % Change": f"±{int(pct * 100)}%",
                "Simulated Underlying (+)": round(stock_up, 2),
                "Simulated Underlying (-)": round(stock_down, 2),

                # Simulated Premiums
                "Simulated Premium (+)": round(premium_up, 2),
                "Simulated Premium (-)": round(premium_down, 2),

                # Simulated Returns
                "Premium % Change (+)": round(pct_up_gain, 2),
                "Premium % Change (-)": round(pct_down_loss, 2)
            })

        return pd.DataFrame(rows)

    except Exception as e:
        logging.error(f"Error simulating option scenario for {ticker}: {e}")
        return pd.DataFrame()


def whole_watchlist(contract_list):
    all_rows = []

    for contract in contract_list:
        df = simulate_option_watchlist_single(
            ticker=contract["ticker"],
            option_type=contract["option_type"],
            strike=contract["strike"],
            expiration=contract["expiration"],
            days_to_gain=contract["days_to_gain"],
            number_of_contracts=contract["number_of_contracts"],
            average_cost_per_contract=contract["average_cost_per_contract"]
        )

        if not df.empty:
            all_rows.append(df)

    return pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()


In [4]:
df = simulate_option_watchlist_single(
    ticker="AAPL",
    option_type="call",
    strike=220.0,
    expiration="2025-12-19",
    days_to_gain=30,
    number_of_contracts=3,
    average_cost_per_contract=4.5
)

display(df)
# df.to_csv("watchlist_scenarios.csv", index=False)


C:\Users\Daniel2\AppData\Local\Temp\ipykernel_61768\282149531.py:38: RuntimeWarning: divide by zero encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
C:\Users\Daniel2\AppData\Local\Temp\ipykernel_61768\282149531.py:38: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))


,Ticker,Option Type,Strike,Expiration,Implied Volatility,Number of Contracts,Average Cost per Contract,Average Cost ($),Current Premium,Your Equity ($),Total % Change,Days to Gain,Scenario %,Stock Price Up,Stock Price Down,Premium Up,Premium Down,Premium % Gain (Up),Premium % Loss (Down)
0,AAPL,call,220.0,2025-12-19,26.25,3,4.5,1350.0,8.8,2640.0,95.56,30,±5%,211.05,190.95,11.82,4.46,34.26,-49.29
1,AAPL,call,220.0,2025-12-19,26.25,3,4.5,1350.0,8.8,2640.0,95.56,30,±10%,221.10,180.90,17.21,2.38,95.55,-72.97
2,AAPL,call,220.0,2025-12-19,26.25,3,4.5,1350.0,8.8,2640.0,95.56,30,±20%,241.20,160.80,31.07,0.46,253.03,-94.80
3,AAPL,call,220.0,2025-12-19,26.25,3,4.5,1350.0,8.8,2640.0,95.56,30,±50%,301.50,100.50,86.12,0.00,878.69,-100.00
4,AAPL,call,220.0,2025-12-19,26.25,3,4.5,1350.0,8.8,2640.0,95.56,30,±100%,402.00,0.00,186.30,0.00,2017.02,-100.00
5,AAPL,call,220.0,2025-12-19,26.25,3,4.5,1350.0,8.8,2640.0,95.56,30,±200%,603.00,-201.00,387.30,NaN,4301.10,NaN


In [5]:
# # bought around march 26, 2025

# watchlist = [
#     {
#         "ticker": "MSFT",
#         "option_type": "call",
#         "strike": 500.0,
#         "expiration": "2025-12-19",
#         "days_to_gain": 30,
#         "number_of_contracts": 3,
#         "average_cost_per_contract": 5.85
#     },
#     {
#         "ticker": "AAPL",
#         "option_type": "call",
#         "strike": 220.0,
#         "expiration": "2025-12-19",
#         "days_to_gain": 30,
#         "number_of_contracts": 3,
#         "average_cost_per_contract": 25.23
#     }
# ]

# watchlist_df = whole_watchlist(watchlist)
# display(watchlist_df)
# watchlist_df.to_csv("watchlist_output.csv", index=False)
